# Neural Ordinary Differential Equations 神经常微分方程

[翻译自原文](https://nbviewer.org/github/urtrial/neural_ode/blob/master/Neural%20ODEs.ipynb)

**过程**很大策程度上可以用微分方程来描述:可以是物理系统的演化、病人的医疗条件、市场的基本属性等。这些数据在本质上是序列化并连续的，这意味着对它们的观测仅仅是某些不断变化的状态的具象化表示。

还有另一种类型的序列数据是离散的——例如NLP数据:它的状态是离散的，从一个符号到另一个符号，从一个单词到另一个单词。

今天，这两种类型数据的处理通常使用循环神经网络。然而，它们在本质上是不同的，似乎应该区别对待。

NIPS2018有一篇非常有趣的[论文](https://arxiv.org/abs/1806.07366)试图解决这个问题。作者提出了一个非常有前途的方法，他们称之为**神经常微分方程**。

在这里，我试图复制和总结原始论文的结果，使你更容易熟悉这个想法。我相信，在卷积和循环网络中，这种新的结构可能很快就会出现在很多数据科学家的工具箱中。

想象一个问题:有一个过程跟随一个未知的ODE，并随着时间有一些关于这个过程的(有噪声的)观测
$$ \frac{dz}{dt} = f(z(t), t) \tag{1} $$$$ \{(z_0, t_0),(z_1, t_1),...,(z_M, t_M)\} - \text{observations} $$
那是否有可能找到$f(z, t)$的近似$\widehat{f}(z, t, \theta)$呢？